### 当前实验模型内容

1. use mentions
2. 63.3

3. add passage score

model_name |  param | dev_acc|
---| --- | ---
use mentions | lr=1e-3,hidden=50 | 63.3
um-ps | 5e-4, 50 dropout:0.2| 64.94
um-ps | reson_p:0.2, step=3 | 65.00
um-ps | reason p:0.4, step=5| 66.12
um-ps no n-gram char| reason p:0.4, step=5| 65.50

In [1]:
import os
import torch
import torch.nn as nn
import torchtext
from tensorboardX import SummaryWriter
import random
import numpy as np

from torchtext.data import NestedField, Field, RawField
from model import *
from dataset import DataHandler
%load_ext autoreload

%autoreload 2

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
class Config:
    def __init__(self):
        self.hidden = 50
        self.embedding_dim = 300
        self.lr = 5e-4
        self.epochs = 50
        self.fix_length = None
        
        self.log_dir = './logs'
        self.model_name = 'CFC_um_ps_no_char_save'
        self.batch_size = 4
        self.train_data = './data/train_filter.pt'
        self.dev_data = './data/dev_filter.pt'
        
        self.word_vocab = './data/glove_vocab.pt'
        self.charNGram_vocab = './data/charNGram_vocab.pt'
        #self.word_vocab = None
        #self.charNGram_vocab = None
        
        self.dropout = 0.2
        self.seed = 1023
        self.steps = 5
        
config = Config()
device = torch.device("cuda:0")


In [3]:
torch.cuda.is_available()

True

In [4]:
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)

In [5]:
save_path = config.model_name  + '_lr_'+ str(config.lr)+ '__hidden__' + str(config.hidden) \
            + '_batchsize_' + str(config.batch_size) +  '_p'+ str(config.dropout)+'_steps'+str(config.steps)+'cycle_lr'
save_path = os.path.join(config.log_dir, save_path)   
print(save_path)
config.save_path = save_path

./logs/CFC_um_ps_no_char_save_lr_0.0005__hidden__50_batchsize_4_p0.2_steps5cycle_lr


### Define Fileds

In [6]:
word_field = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True) # query
multi_word_field = NestedField(word_field) 

word_field_sup = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True, fix_length=config.fix_length)
multi_word_field_sup = NestedField(word_field_sup) 

charNGram_field = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True) # query
multi_charNGram_field = NestedField(charNGram_field) 

charNGram_field_sup = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True, fix_length=config.fix_length)
multi_charNGram_field_sup = NestedField(charNGram_field_sup) 

raw = RawField()
raw.is_target = False

label_field = Field(sequential=False, is_target=True, use_vocab=False)

dict_field = {
    'id': ('id', raw),
    'supports': [('s_glove', multi_word_field_sup), ('s_charNGram', multi_charNGram_field_sup)],
    'query': [('q_glove', word_field), ('q_charNGram', charNGram_field)],
    'candidates': [('c_glove', multi_word_field), ('c_charNGram', multi_charNGram_field)],
    'label': ('label', label_field),
    'mentions': ('mentions', raw),
    'para_label': ('para_label', raw)
}

In [7]:
data_handler = DataHandler(config.train_data, config.dev_data, dict_field)

# torch.save(data_handler.trainset.examples, './data/train_example.pt')
# torch.save(data_handler.valset.examples, './data/dev_example.pt')

load examples.pt  :./data/train_filter.pt, ./data/dev_filter.pt


### Build Vocab

In [8]:
if config.charNGram_vocab is not None:
    charNGram_vocab = torch.load(config.charNGram_vocab)
    multi_charNGram_field_sup.vocab = charNGram_vocab
    charNGram_field_sup.vocab = charNGram_vocab
else:
    multi_charNGram_field_sup.build_vocab(data_handler.trainset, data_handler.valset, 
                                          vectors=torchtext.vocab.CharNGram())

if config.word_vocab is not None:
    word_vocab = torch.load(config.word_vocab)
    multi_word_field_sup.vocab = word_vocab
    word_field_sup.vocab = word_vocab
else:
    multi_word_field_sup.build_vocab(data_handler.trainset, data_handler.valset, 
                                 vectors=torchtext.vocab.GloVe(dim=300,name='6B') )

word_field.vocab = multi_word_field_sup.vocab
charNGram_field.vocab = multi_charNGram_field_sup.vocab



In [9]:
print(multi_word_field_sup.vocab.vectors.shape,multi_charNGram_field_sup.vocab.vectors.shape )

torch.Size([312667, 300]) torch.Size([312667, 100])


### Get data_iter

In [10]:
train_iter = data_handler.get_train_iter(batch_size=config.batch_size)
val_iter = data_handler.get_val_iter(batch_size=config.batch_size)

In [11]:
for idx, batch in enumerate(val_iter):
    break
batch


[torchtext.data.batch.Batch of size 4]
	[.id]:['WH_dev_0', 'WH_dev_1', 'WH_dev_2', 'WH_dev_3']
	[.s_glove]:[torch.LongTensor of size 4x15x292]
	[.s_charNGram]:[torch.LongTensor of size 4x15x292]
	[.q_glove]:[torch.LongTensor of size 4x11]
	[.q_charNGram]:[torch.LongTensor of size 4x11]
	[.c_glove]:[torch.LongTensor of size 4x18x4]
	[.c_charNGram]:[torch.LongTensor of size 4x18x4]
	[.label]:[torch.LongTensor of size 4]
	[.mentions]:[[[[6, 145, 146], [6, 173, 174], [7, 78, 79]], [[3, 50, 53], [5, 28, 31], [13, 1, 4]], [[6, 135, 136], [6, 218, 219], [6, 261, 262], [8, 45, 46], [12, 98, 99]], [[0, 2, 4], [7, 1, 3], [13, 64, 66], [13, 69, 71]], [[0, 36, 38], [10, 1, 3], [13, 75, 77]], [[0, 14, 15], [1, 63, 64], [1, 138, 139], [1, 186, 187], [1, 238, 239], [2, 128, 129], [9, 8, 9], [9, 43, 44], [10, 19, 20], [10, 34, 35], [11, 37, 38], [11, 79, 80], [13, 56, 57]], [[7, 37, 40]], [[6, 180, 181], [12, 101, 102]], [[12, 96, 97]], [[8, 43, 46]], [[6, 169, 172]], [[6, 179, 181]], [[7, 38, 40]], 

In [12]:
def get_para_label(batch):
    para_size = batch.s_glove.size(1)
    results = []
    for label in batch.para_label:
        padding = [0]*(para_size - len(label))
        n_label = label[:]
        n_label += padding
        results.append(n_label)
    results = torch.tensor(results, dtype=torch.long)
    return results

### Define Model

In [13]:
def generate_mask(x_size, num_turn, dropout_p=0.0, is_training=False):
    if not is_training: dropout_p = 0.0
    new_data = torch.zeros(x_size, num_turn)
    new_data = (1-dropout_p) * (new_data.zero_() + 1)
    for i in range(new_data.size(0)):
        one = random.randint(0, new_data.size(1)-1)
        new_data[i][one] = 1
    mask = 1.0/(1 - dropout_p) * torch.bernoulli(new_data)
    mask.requires_grad = False
    return mask

class SAN(nn.Module):
    def __init__(self, x_size, h_size, c_size, num_turn=5, dropout=0.2, memo_dropout=0.4, device=None):
        super(SAN,self).__init__()
        self.att = BilinearSeqAttn(x_size, h_size, dropout=dropout)
        self.rnn = nn.GRUCell(x_size, h_size)
        self.num_turn = num_turn
        self.fc = nn.Linear(x_size, c_size)
        self.dropout = nn.Dropout(p=dropout)
        self.memo_dropout=memo_dropout
        self.device = device
        
    def forward(self, x, h0, c):
        '''
        x: [batch, sup_len, hidden]
        h0:[batch, hidden]
        c: [batch, can_len, hidden]
        '''
        score_list = []
        
        for turn in range(self.num_turn):
            score = self.att(x, h0)
            x_att = torch.bmm(F.softmax(score,1).unsqueeze(1),x).squeeze(1) # [batch, hidden]
            
            h0 = self.dropout(h0)
            h0 = self.rnn(x_att, h0)
            
            x_att = torch.tanh(self.fc(x_att)) 

            score = torch.bmm(c, x_att.unsqueeze(-1))
            score = score.squeeze(-1) # [batch, can_len]
            score_list.append(score)
            
        mask = generate_mask(x.size(0),self.num_turn, self.memo_dropout, self.training)
        mask = mask.to(self.device)
        mask = [m.contiguous() for m in torch.unbind(mask, 1)]
        
        score_list = [mask[idx].view(x.size(0), 1).expand_as(inp) * inp for idx, inp in enumerate(score_list)]
        scores = torch.stack(score_list, 2)
        scores = torch.mean(scores, 2)
        return scores
    
    

In [14]:
class SimpleQANet(nn.Module):
    
    def __init__(self, config, word_vectors, charNGram_vectors, device):
        super(SimpleQANet, self).__init__()
        self.config = config
        self.device = device
        
        self.embedding_layer = EmbeddingLayer(word_vectors, charNGram_vectors)
        

        self.rnn = EncoderRNN(config.embedding_dim, config.hidden, 1, True, True, config.dropout, False)
                
            
        self.co_att = CoAttention(config.hidden*2, att_type=2, dropout=config.dropout)
        
        self.linear_1 = nn.Sequential(
                        nn.Linear(config.hidden*4, config.hidden),
                        nn.ReLU()
                    )        
        self.rnn2 =  EncoderRNN(config.hidden, config.hidden, 1, True, True, config.dropout, False)
        
        self.word_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        self.word_att_q = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        self.p_score = BilinearSeqAttn(config.hidden*2, config.hidden*2, identity=False, dropout=config.dropout)
        
        self.pass_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        self.c_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        self.mention_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        
        #self.fusion = FusionLayer(config.hidden*2, dropout=config.dropout)
        self.max_pooling = PoolingLayer()     
        
        self.fc = nn.Linear(config.hidden*2, config.hidden*4)
        self.san = SAN(config.hidden*2,config.hidden*2,config.hidden*4, num_turn=config.steps, device=device)
        
        self.to(device)
        
    def forward(self, batch, return_label = True):
        if type(batch.q_glove) is tuple:
            q_glove, _ = batch.q_glove
            q_charNGram, _ = batch.q_charNGram
        else:
            q_glove = batch.q_glove
            q_charNGram = batch.q_charNGram            
        
        s_glove = batch.s_glove
        s_charNGram = batch.s_charNGram
        
        c_glove = batch.c_glove
        c_charNGram = batch.c_charNGram
        
        q_glove = q_glove.to(self.device)
        q_charNGram = q_charNGram.to(self.device)

        s_glove = s_glove.to(self.device)
        s_charNGram = s_charNGram.to(self.device)

        c_glove = c_glove.to(self.device)
        c_charNGram = c_charNGram.to(self.device)
        
        ### Embedding and Encoder
        
        #q_out = self.embedding_layer(q_glove, q_charNGram)
        #s_out = self.embedding_layer(s_glove, s_charNGram,)
        #c_out = self.embedding_layer(c_glove, c_charNGram)
        
        q_out = self.embedding_layer(q_glove)
        s_out = self.embedding_layer(s_glove)
        c_out = self.embedding_layer(c_glove)        
        
        batch_size=  s_out.size(0)
        
        s_len = s_out.size(1)
        c_len = c_out.size(1)
        
        s_word_len = s_out.size(2)
        c_word_len = c_out.size(2)
        
        hidden = s_out.size(-1)
        
        s_out = s_out.view(batch_size*s_len, s_word_len, hidden).contiguous()
        c_out = c_out.view(batch_size*c_len, c_word_len, hidden).contiguous()
        
        q_out = self.rnn(q_out)
        c_out = self.rnn(c_out)
        s_out = self.rnn(s_out)
        
        # Attention
        
        q_word_len = q_out.size(1)
        q_out_expand = q_out.unsqueeze(1).expand(batch_size, s_len, q_word_len, q_out.size(-1)).contiguous()
        q_out_expand = q_out_expand.view(batch_size*s_len, q_word_len, q_out.size(-1)).contiguous()
        
        s_out_att, q_out_att = self.co_att(s_out, q_out_expand)
        #S_s = self.fusion(s_out, s_out_att)
        #S_q = self.fusion(q_out, q_out_att)
        
        S_s = self.linear_1(s_out_att)
        S_s = self.rnn2(S_s)
        
        
        
        batch_c_m = []
        for i in range(batch_size):
            # get mention embedding
            mentions = batch.mentions[i]
            c_ms = torch.zeros(c_len, s_len, s_out.size(-1))
            for idx, c_mention in enumerate(mentions):
                c_m_dict = {}
                for mention in c_mention:
                    m = s_out[i*s_len + mention[0]][mention[1]:mention[2]]
                    m = self.max_pooling(m.unsqueeze(0)).squeeze()
                    if mention[0] not in c_m_dict:
                        c_m_dict[mention[0]] = []
                    c_m_dict[mention[0]].append(m)
                c_m = torch.zeros(s_len, s_out.size(-1))
                for key in c_m_dict:
                    for m in c_m_dict[key]:
                        c_m[key] += m.cpu()
                    c_m[key] /= len(c_m_dict[key])
                c_ms[idx] = c_m
            batch_c_m.append(c_ms)
        batch_c_m = torch.stack(batch_c_m)
        batch_c_m = batch_c_m.to(self.device)
        batch_c_m = batch_c_m.view(batch_size*c_len, s_len, -1)
        batch_c_m = self.mention_att(batch_c_m)
        batch_c_m = batch_c_m.view(batch_size, c_len, -1)
        

        
        C_s = self.word_att(S_s)
        C_q = self.word_att_q(q_out)
        
        C_s = C_s.view(batch_size, s_len, -1)
        
        P_scores = self.p_score(C_s, C_q)
        

        
        C_c = self.c_att(c_out)        
        C_c = C_c.view(batch_size, c_len, -1)
        
        C_c = torch.cat([C_c, batch_c_m],-1)
        
        
        score = self.san(C_s, C_q, C_c)
        
        if return_label:
            label = batch.label.to(self.device)
            P_label = get_para_label(batch)
            P_label = P_label.to(device)
            return score, P_scores, label, P_label
        return score, P_scores

#### test model

In [15]:
model = SimpleQANet(config, word_field.vocab.vectors, charNGram_field.vocab.vectors, device)
#score,P_score, label, P_label = model(batch)
#print(score, label, P_score, P_label)
#print(score.shape, label.shape, P_score.shape, P_label.shape)

In [17]:
from tqdm import tqdm, trange

In [18]:
from utils import AverageMeter

def train(epoch, data_iter, model, criterion, criterion_bce, optimizer, batch_size=1, joint_begin=-1):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()
    #model.embedding_layer.eval()
    with trange(len(data_iter)) as t:
        for idx, batch in enumerate(data_iter):
            score, P_score, label, P_label = model(batch)

            loss1 = criterion(score, label)
            loss_p = criterion_bce(P_score, P_label.float())
            if epoch > joint_begin:
                loss = loss1 + loss_p     
            else:
                loss = loss_p

            loss = loss / batch_size
            loss.backward()
            if (idx+1)%batch_size == 0 :
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)            
                optimizer.step()
                optimizer.zero_grad()        

            losses.update(loss.item()*batch_size)

            pred = score.argmax(1)
            acc = pred.eq(label).sum().item()  / pred.size(0)
            acces.update(acc)
            
            t.update()
            if (idx+1) % (batch_size*100) == 0:
                print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

def val(epoch, data_iter, model, criterion, criterion_bce, joint_begin=-1):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()
    for idx, batch in enumerate(data_iter):
        with torch.no_grad():
            score, P_score, label, P_label = model(batch)
                    
        loss1 = criterion(score, label)
        loss_p = criterion_bce(P_score, P_label.float())
        if epoch > joint_begin:
            loss = loss1 + loss_p     
        else:
            loss = loss_p
        
        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()  / pred.size(0)
        acces.update(acc)
        if idx % 100 == 0:
            print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

In [19]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                             lr=config.lr)

criterion = nn.CrossEntropyLoss()
criterion_bce = nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs)
#train(0, train_iter, model, criterion, optimizer, batch_size=config.batch_size)
# val(0, val_iter, model,criterion)

In [20]:
cycle_len = 1
cycle_iter = 50

In [ ]:
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
writer = SummaryWriter(config.save_path)

best_acc = 0.0
for i in range(cycle_len):
    optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                             lr=config.lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cycle_iter)
    for epoch in range(cycle_iter):
        scheduler.step()
        train_loss, train_acc = train(epoch, train_iter, model, criterion, criterion_bce, optimizer, 
                                         1)
        val_loss, val_acc = val(epoch, val_iter, model, criterion, criterion_bce)
        global_epoch = cycle_iter * i + epoch + 1
        writer.add_scalar('train_loss', train_loss, global_epoch)
        writer.add_scalar('val_loss', val_loss, global_epoch)
        writer.add_scalar('train_acc', train_acc, global_epoch)
        writer.add_scalar('val_acc', val_acc, global_epoch)

        state = {
            'val_acc': val_acc,
            'train_acc': train_acc,
            'epoch': epoch,
            'model': model.state_dict()
        }
        torch.save(state, os.path.join(config.save_path,'lastest.pth'))
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(state, os.path.join(save_path, f'best_epoch{epoch}.pth'))

  0%|          | 0/10845 [00:00<?, ?it/s]/home/zzs/Multi_Evidence_QA/model.py:186: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)
  1%|          | 100/10845 [01:02<1:32:44,  1.93it/s]

epoch:0, idx:99/10845, loss:3.5322713935375214, acc:0.19


  2%|▏         | 200/10845 [02:00<1:35:07,  1.87it/s]

epoch:0, idx:199/10845, loss:3.1706510162353516, acc:0.25


  3%|▎         | 300/10845 [03:01<1:39:05,  1.77it/s]

...

  8%|▊         | 900/10845 [08:57<1:49:30,  1.51it/s]

epoch:2, idx:899/10845, loss:1.5713239905238152, acc:0.5925


  9%|▉         | 1000/10845 [10:00<1:31:02,  1.80it/s]

epoch:2, idx:999/10845, loss:1.5715386326909064, acc:0.59225


  9%|▉         | 1020/10845 [10:11<1:39:25,  1.65it/s]